<a href="https://colab.research.google.com/github/Marytem/practice_2019/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

__need__:

accuracy

__would be good__

grid search

__experiment__

optimizer=ks.optimizers.SGD(lr=0.01)

<!-- try l2 regularizattion
 -->
learning rate

more layers?

### data mixing & preprocessing

In [0]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


In [5]:
gen_target = pd.read_csv('bbc_target.csv', header=None)[0].append(pd.Series(data=fetch_20newsgroups(subset='all').target), ignore_index=True)
gen_text = pd.read_csv('bbc_text.csv', header=None)[0].append(pd.Series(data=fetch_20newsgroups(subset='all', remove=('headers', 'footers')).data), ignore_index=True)

In [0]:
gen_target.replace({'business':20, 'entertainment':21, 'politics':22, 'sport':23, 'tech':24 }, inplace=True)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(gen_text, gen_target)

In [0]:
tfidf = TfidfVectorizer(sublinear_tf=True, norm='l2', stop_words='english', min_df=8)
features = tfidf.fit_transform(x_train)

In [10]:
features.shape

(15802, 19671)

In [0]:
# ['alt.atheism',0,0
#  'comp.graphics',1,1
#  'comp.os.ms-windows.misc',2,1
#  'comp.sys.ibm.pc.hardware',3,1
#  'comp.sys.mac.hardware',4,1
#  'comp.windows.x',5,1
#  'misc.forsale',6,1
#  'rec.autos',7,0
#  'rec.motorcycles',8,0
#  'rec.sport.baseball',9.0
#  'rec.sport.hockey',10,0
#  'sci.crypt',11,1
#  'sci.electronics',12,1
#  'sci.med',13,1
#  'sci.space',14,1
#  'soc.religion.christian',15,0
#  'talk.politics.guns',16,0
#  'talk.politics.mideast',17,0
#  'talk.politics.misc',18,0
#  'talk.religion.misc',19,0]
#  'business',20, 1
#  'entertainment',21, 0
#  'politics',22, 0
#  'sport',23, 0
#  'tech',24, 1

tech = [1,2,3,4,5,6,11,12,13,14,20,24]


In [0]:
bin_tr_y = np.isin(y_train.values, tech).astype(np.uint8)
bin_tes_y = np.isin(y_test.values, tech).astype(np.uint8)

In [13]:
vectors_test = tfidf.transform(x_test)
# np.savetxt('test_x', vectors_test.toarray()[:2])
clf = MultinomialNB(alpha=.01)
clf.fit(features, bin_tr_y)
pred = clf.predict(vectors_test)
accuracy_score(bin_tes_y, pred)

0.9538724373576309

In [14]:
unique, counts = np.unique(bin_tr_y, return_counts=True)
dict(zip(unique, counts))

{0: 8476, 1: 7326}

### Model & train

In [15]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
import keras

Using TensorFlow backend.


In [0]:
simple_model = Sequential([Dense(100,activation='relu', input_dim=features.shape[1], kernel_regularizer=l2(0.01)),
                           Dropout(0.5, noise_shape=None, seed=None),
                           Dense(100,activation='relu', kernel_regularizer=l2(0.01)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(1,activation='sigmoid')])

ES_callback = EarlyStopping(monitor='val_loss', min_delta=0.00005, patience=4)

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-{acc:.4f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

In [0]:
simple_model.compile(loss='binary_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])

In [18]:
simple_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               1967200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 1,977,401
Trainable params: 1,977,401
Non-trainable params: 0
_________________________________________________________________


In [19]:
model_output= simple_model.fit(features,
                               bin_tr_y,
                               batch_size=256,
                               epochs=40,
                               verbose=1,
                               validation_data=(vectors_test, bin_tes_y),
                               callbacks=[ES_callback, checkpoint])

Train on 15802 samples, validate on 5268 samples
Epoch 1/40
15802/15802 [==============================] - 5s 333us/step - loss: 1.4827 - acc: 0.6602 - val_loss: 0.9558 - val_acc: 0.8565

Epoch 00001: loss improved from inf to 1.48270, saving model to weights-improvement-01-1.4827-0.6602.hdf5
Epoch 2/40
15802/15802 [==============================] - 3s 181us/step - loss: 0.7970 - acc: 0.8825 - val_loss: 0.6653 - val_acc: 0.9330

Epoch 00002: loss improved from 1.48270 to 0.79701, saving model to weights-improvement-02-0.7970-0.8825.hdf5
Epoch 3/40
15802/15802 [==============================] - 3s 181us/step - loss: 0.6233 - acc: 0.9160 - val_loss: 0.5662 - val_acc: 0.9339

Epoch 00003: loss improved from 0.79701 to 0.62331, saving model to weights-improvement-03-0.6233-0.9160.hdf5
Epoch 4/40
15802/15802 [==============================] - 3s 180us/step - loss: 0.5623 - acc: 0.9234 - val_loss: 0.5332 - val_acc: 0.9334

Epoch 00004: loss improved from 0.62331 to 0.56233, saving model to w

#### Saving the most successful weighted model

In [0]:
simple_model = Sequential([Dense(100,activation='relu', input_dim=features.shape[1], kernel_regularizer=l2(0.01)),
                           Dropout(0.5, noise_shape=None, seed=None),
                           Dense(100,activation='relu', kernel_regularizer=l2(0.01)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(1,activation='sigmoid')])

simple_model.load_weights('cudnn-overfitting-bigdata-weights-improvement-14-0.4001-0.9642.hdf5')

simple_model.compile(loss='binary_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])

In [0]:
!pip install tensorflowjs

    15% |████▉                           | 2.1MB 41.3MB/s eta 0:00:01
Operation cancelled by user


In [0]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(simple_model, 'model_stillnotfat')

#### Manual trial

In [0]:
# text = 'i want to use this classifier for a commercial project. It is light-weighted and can work in a chrome plugin'
# text = 'i care for my country very much, vote for me. believeme, i will be the best president you have ever known'
# text = 'This ballet style is often performed barefoot. Contemporary ballets may include mime and acting, and are usually set to music (typically orchestral but occasionally vocal). It can be difficult to differentiate this form from neoclassical or modern ballet. Contemporary ballet is also close to contemporary dance, because many contemporary ballet concepts come from the ideas and innovations of twentieth-century modern dance, including floor work and turn-in of the legs. The main distinction is that ballet technique is essential to perform a contemporary ballet.George Balanchine is considered to have been a pioneer of contemporary ballet. Another early contemporary ballet choreographer, Twyla Tharp, choreographed Push Comes To Shove for the American Ballet Theatre in 1976, and in 1986 created In The Upper Room for her own company. Both of these pieces were considered innovative for their melding of distinctly modern movements with the use of pointe shoes and classically trained dancers.Today there are many contemporary ballet companies and choreographers. These include Alonzo King and his company LINES Ballet; Matthew Bourne and his company New Adventures; Complexions Contemporary Ballet; Nacho Duato and his Compañia Nacional de Danza; William Forsythe and The Forsythe Company; and Jiří Kylián of the Nederlands Dans Theater. Traditionally "classical" companies, such as the Mariinsky (Kirov) Ballet and the Paris Opera Ballet, also regularly perform contemporary works.The term ballet has evolved to include all forms associated with it. Someone training as a ballet dancer will now be expected to perform neoclassical, modern and contemporary work. A ballet dancer is expected to be able to be stately and regal for classical work, free and lyrical in neoclassical work, and unassuming, harsh or pedestrian for modern and contemporary work. In addition, there are several modern varieties of dance that fuse classical ballet technique with contemporary dance, such as Hiplet, that require dancers to be practised in non-Western dance styles.'
# text='The first thing that comes into our minds while thinking of employees at tech companies is probably Software Engineers and Data Scientists. But a technology company requires employees beyond those in technical roles, so they hire for non-tech roles too. The current scenario at the job market doesn’t necessarily require you to be a tech pro to get hired by technology companies. As the growth tech companies increases, they need more non-technical workforce to help transform their technology into revenue. Roles such as sales, marketing, business operation, etc. are crucial to their expansion. These highly coveted positions are also well compensated by tech employers.'
text = '''As part of an effort to reduce noise in Ghana’s capital, Accra, the environment minister has suggested that the Muslim call to prayer, normally broadcast over loudspeakers across the city, should instead be sent out on WhatsApp. The notion has proved immensely unpopular – not least because it equates the call to prayer with noise pollution. But it also highlights religion’s growing, if sometimes uneasy, reliance on tech.

Contactless collection Catholic and Protestant churches in the UK have begun using contactless card readers for donations and other payments, hoping to make life easier for parishioners who may not be carrying cash. A contactless collection plate is being trialled by the Church of England, but it is being held up because it is feared the technology might slow things down.

The confession app Confession (version 2.1) walks sinners through the business of confession, pings you push notifications when it’s time for your next shriving and includes a handy sin checklist in case you have forgotten what you’ve done wrong. What it doesn’t do is offer absolution. You still need a priest for that.

Televangelist tech American TV preachers tend to be at home in the digital realm, particularly those espousing “prosperity theology”, which holds that God rewards the pious with material wealth. It works for them, anyway. Controversial televangelist Joel Osteen’s free app offers daily devotional messages and sermons. And Creflo Dollar’s latest app boasts a “give” feature that makes handing your money over to Creflo A Dollar Ministries as easy as gambling. Webcast rites Funerals broadcast online are at least a decade old. Many UK crematoria now offer it as part of the package, and rabbis and vicars routinely officiate at live-streamed services. It is not quite the same as being there, but in many cases it’s probably preferable.Halal apps There are several apps that help Muslims find halal restaurants and stores in the immediate area . There is even one – Scan Halal – that allows you to scan the barcode of any food in a supermarket to instantly determine whether it contains non-halal ingredients. Online reviews for this idea are decidedly mixed.'
'''



In [0]:

transformed = tfidf.transform([text])

In [0]:

simple_model.predict(transformed)


array([[0.20854598]], dtype=float32)

#### Save data for further tf-idf vectorization

In [0]:
open('tfidf_vocab.json', 'w').write(str(tfidf.vocabulary_))

In [0]:
np.savetxt('idf.csv', tfidf.idf_, delimiter=',')

In [0]:
dictionary = tfidf.vocabulary_
dictionary

In [0]:
idf=tfidf.idf_

In [0]:
new_tfidf = TfidfVectorizer(sublinear_tf=True, norm='l2', ngram_range=(1, 2), stop_words='english', min_df=8, vocabulary=dictionary)

In [0]:
new_tfidf.idf_ = idf

In [0]:
transformed = new_tfidf.transform([text])

In [0]:
transformed.shape

(1, 33833)

#### Grid search

### categorical classification trial

In [0]:
from keras.utils import to_categorical
train_target = to_categorical(train.target)
test_target = to_categorical(test.target)

In [0]:
simple_model_cat = Sequential([Dense(100,activation='relu', input_dim=1186545, kernel_regularizer=l2(0.01)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(100,activation='relu', kernel_regularizer=l2(0.01)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(1,activation='softmax')])

In [0]:
simple_model_cat.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
simple_model_cat.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               118654600 
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 118,664,801
Trainable params: 118,664,801
Non-trainable params: 0
_________________________________________________________________


In [0]:
model_output= simple_model_cat.fit(features, train.target, epochs=5, verbose=1,
                               validation_data=(vectors_test, test.target))